###### 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。
（例）
重みを初期化する必要があった
エポックのループが必要だった
それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

1. 初期化メソッドの選択
2. 重み更新メソッドの選択
3. 活性化関数の選択
4. 全結合層
5. エポック数分ループを回す
6. 出力層の選択
7. ノード数の指定

###### 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。
それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

各メソッドはtensorflowライブラリで用意せれており、必要なものをインスタンスの様に定義し使用している。  
各層の結合は、前の層のインスタンス？を次の層のインスタンス？の引数として渡している。  
エポック数分のループはこれまでと同じ。
ノード数もこれまで同様に重み行列の次元に反映される。

In [7]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import tensorflow as tf

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)

# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 24.3878, val_loss : 53.8705, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 16.5814, val_loss : 10.1158, acc : 0.250, val_acc : 0.625
Epoch 2, loss : 3.0246, val_loss : 10.7464, acc : 0.750, val_acc : 0.438
Epoch 3, loss : 0.0893, val_loss : 3.3857, acc : 1.000, val_acc : 0.875
Epoch 4, loss : 2.9513, val_loss : 12.3798, acc : 0.750, val_acc : 0.438
Epoch 5, loss : 0.0002, val_loss : 3.0059, acc : 1.000, val_acc : 0.875
Epoch 6, loss : 0.0000, val_loss : 4.5717, acc : 1.000, val_acc : 0.688
Epoch 7, loss : 0.0000, val_loss : 0.9316, acc : 1.000, val_acc : 0.938
Epoch 8, loss : 0.0000, val_loss : 0.2068, acc : 1.000, val_acc : 0.938
Epoch 9, loss : 0.0000, val_loss : 0.0013, acc : 1.000, val_acc : 1.000
test_acc : 0.900


###### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Iris Species
サンプルコードと同じくこの中のtrain.csvを使用してください。目的変数はSpeciesに含まれる3種類全てを使います。
2クラスの分類と3クラス以上の分類の違いを考慮してください。
それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [36]:
from sklearn.preprocessing import OneHotEncoder

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)

# データフレームから条件抽出
#df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2
#y = y.astype(np.int)[:, np.newaxis]

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:,np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [40]:
y_train.shape

(96, 3)

In [24]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [42]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 39.0804, val_loss : 37.2155, acc : 0.167, val_acc : 0.375
Epoch 1, loss : 0.4878, val_loss : 1.8554, acc : 0.833, val_acc : 0.708
Epoch 2, loss : 5.4925, val_loss : 7.0450, acc : 0.500, val_acc : 0.708
Epoch 3, loss : 2.0867, val_loss : 2.5996, acc : 0.833, val_acc : 0.792
Epoch 4, loss : 0.0101, val_loss : 0.1239, acc : 1.000, val_acc : 0.958
Epoch 5, loss : 0.0000, val_loss : 1.9484, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0000, val_loss : 1.3540, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 0.7179, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0000, val_loss : 0.4894, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.0000, val_loss : 1.1522, acc : 1.000, val_acc : 0.917
test_acc : 1.000


###### 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。
House Prices: Advanced Regression Techniques
この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。
分類問題と回帰問題の違いを考慮してください。

In [73]:
import pandas as pd
import numpy as np
data_ori = pd.read_csv("/Users/naoki/Desktop/DIC/kaggle/housing_price/train.csv")

In [74]:
max_num_of_row = len(data_ori)
data_loss_rate = data_ori.isnull().sum() / max_num_of_row * 100

In [75]:
data_drop_5nan_col = data_ori.dropna(axis=1, thresh=max_num_of_row-5)
print("before:{}".format(len(data_ori.columns.values)))
print("after:{}".format(len(data_drop_5nan_col.columns.values)))

before:81
after:63


In [77]:
data_to_use = data_drop_5nan_col.dropna(axis=0)
data_to_use = data_to_use.reset_index(drop=True)

In [78]:
col_featuer = ['OverallQual',
       'GrLivArea',
       'GarageCars',
       'GarageArea',
       'TotalBsmtSF',
       '1stFlrSF',
       'FullBath',
       'TotRmsAbvGrd',
       'YearBuilt',
       'YearRemodAdd',
      ]

col_target = ['SalePrice']

col = col_featuer + col_target

In [79]:
data = data_to_use[col]

In [82]:
X = data[col_featuer].values
Y = data[col_target].values

In [83]:
# trainとtestに分割
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [88]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 30

n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.abs(Y - logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
correct_pred = logits

# 指標値計算
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#決定係数R2
accuracy = 1 - (tf.reduce_sum(tf.square(Y - correct_pred)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 36160.7773, val_loss : 45582.4414, acc : 0.896, val_acc : 0.315
Epoch 1, loss : 37611.1602, val_loss : 34510.3789, acc : 0.927, val_acc : 0.469
Epoch 2, loss : 24136.5215, val_loss : 33705.1367, acc : 0.969, val_acc : 0.409
Epoch 3, loss : 16375.8330, val_loss : 35461.4883, acc : 0.979, val_acc : 0.364
Epoch 4, loss : 18713.4434, val_loss : 32109.1133, acc : 0.981, val_acc : 0.401
Epoch 5, loss : 23763.6934, val_loss : 32548.7871, acc : 0.973, val_acc : 0.442
Epoch 6, loss : 15515.1094, val_loss : 33522.7656, acc : 0.985, val_acc : 0.369
Epoch 7, loss : 15977.7656, val_loss : 32203.9121, acc : 0.985, val_acc : 0.401
Epoch 8, loss : 14114.8281, val_loss : 32179.0430, acc : 0.986, val_acc : 0.397
Epoch 9, loss : 16170.6201, val_loss : 31774.9707, acc : 0.982, val_acc : 0.416
Epoch 10, loss : 21064.6250, val_loss : 31425.0605, acc : 0.978, val_acc : 0.469
Epoch 11, loss : 19939.7031, val_loss : 30350.0020, acc : 0.974, val_acc : 0.342
Epoch 12, loss : 16096.7139, val_loss 

###### 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。
3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。
スクラッチで実装したモデルの再現を目指してください。

In [89]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype(np.float)
x_test = x_test.astype(np.float)
x_train /= 255
x_test /= 255
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [90]:
#train , val
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train_one_hot, test_size=0.4)
print(x_train.shape) # (48000, 784)
print(x_val.shape) # (12000, 784)

(36000, 784)
(24000, 784)


In [91]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 3.6943, val_loss : 1.3133, acc : 0.800, val_acc : 0.630
Epoch 1, loss : 1.0561, val_loss : 1.1360, acc : 0.700, val_acc : 0.664
Epoch 2, loss : 1.3494, val_loss : 0.8586, acc : 0.800, val_acc : 0.776
Epoch 3, loss : 0.3350, val_loss : 0.6679, acc : 0.900, val_acc : 0.825
Epoch 4, loss : 0.7205, val_loss : 0.5735, acc : 0.800, val_acc : 0.872
Epoch 5, loss : 0.6452, val_loss : 0.4503, acc : 0.900, val_acc : 0.882
Epoch 6, loss : 0.7340, val_loss : 0.4110, acc : 0.800, val_acc : 0.893
Epoch 7, loss : 1.2474, val_loss : 0.4907, acc : 0.900, val_acc : 0.877
Epoch 8, loss : 0.6046, val_loss : 0.3936, acc : 0.900, val_acc : 0.898
Epoch 9, loss : 0.4634, val_loss : 0.3981, acc : 0.900, val_acc : 0.907
test_acc : 0.911


In [92]:
tf.summary.FileWriter('val_loss', sess.graph)
!tensorboard --logdir=val_loss --port=8010  #ポート指定できます

TensorBoard 1.12.2 at http://sasakinaokinoMacBook-puro.local:8010 (Press CTRL+C to quit)
^C


In [94]:
tf.summary.FileWriter('val_loss', sess.graph)
!tensorboard --logdir=val_loss --port=8080  #ポート指定できます

TensorBoard 1.12.2 at http://sasakinaokinoMacBook-puro.local:8080 (Press CTRL+C to quit)
^C
